In [ ]:
# Importing required libraries
import pandas as pd
import os

# File path to the data (update this with your actual file path)
file_path = "./MTM04.20241212173447.csv"  # Adjust path to match the actual location of your CSV file

# Checking if the file exists
if os.path.exists(file_path):
    try:
        # Loading the data into a pandas DataFrame
        wind_data = pd.read_csv(file_path)

        # Displaying the first few rows of the dataset
        print("Data loaded successfully. Here are the first 5 rows:")
        print(wind_data.head())

        # Displaying basic information about the dataset
        print("\nDataset Information:")
        print(wind_data.info())

        # Displaying basic statistics of numeric columns
        print("\nSummary Statistics:")
        print(wind_data.describe())

        # Inspecting missing values
        print("\nMissing Values:")
        print(wind_data.isnull().sum())

        # Handling missing values (example: drop rows with missing values)
        wind_data_cleaned = wind_data.dropna()
        print("\nData after dropping missing values:")
        print(wind_data_cleaned.info())

        # Parsing dates if a date column exists (adjusting the column name as needed)
        if 'date' in wind_data_cleaned.columns:
            wind_data_cleaned['date'] = pd.to_datetime(wind_data_cleaned['date'], errors='coerce')
            wind_data_cleaned.set_index('date', inplace=True)
            print("\nDate column parsed and set as index.")

        # Renaming columns for clarity (example, adjust as needed)
        # wind_data_cleaned.rename(columns={"OldColumnName": "NewColumnName"}, inplace=True)

        # Saving the cleaned data to a new CSV file (optional)
        cleaned_file_path = "./cleaned_wind_data.csv"
        wind_data_cleaned.to_csv(cleaned_file_path, index=True)
        print(f"\nCleaned data saved to {cleaned_file_path}")

    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
else:
    print(f"File not found at {file_path}. Please ensure the file path is correct.")


Data loaded successfully. Here are the first 5 rows:
  STATISTIC                        STATISTIC Label  TLIST(M1)    Month  \
0     MTM04  Wind, Maximum Gale Gust (>33.5 Knots)     195801  1958M01   
1     MTM04  Wind, Maximum Gale Gust (>33.5 Knots)     195801  1958M01   
2     MTM04  Wind, Maximum Gale Gust (>33.5 Knots)     195801  1958M01   
3     MTM04  Wind, Maximum Gale Gust (>33.5 Knots)     195801  1958M01   
4     MTM04  Wind, Maximum Gale Gust (>33.5 Knots)     195801  1958M01   

   C02431V02938 Meteorological Weather Station   UNIT  VALUE  
0             1                      Belmullet  Knots   62.0  
1             2                           Birr  Knots   58.0  
2             3           Valentia Observatory  Knots   50.0  
3             4                       Casement  Knots    NaN  
4             5                    Claremorris  Knots   53.0  

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11700 entries, 0 to 11699
Data columns (total 8 colu

### Read in a number of good data sets

- Loaded the dataset (met_eireann_wind_data.csv) into a pandas DataFrame.

- Included error handling to check if the file exists and handle potential issues during loading.


### Analyzed the data using various techniques

- Performed initial inspection: displayed first rows, dataset information (info), summary statistics (describe), and missing values analysis.

- Cleaned the data by handling missing values (e.g., dropping rows with NaN).
Parsed and indexed a date column (if present), preparing the data for time-series analysis.
